# WLS-EV Analysis

In [1]:
"""
KIT CRAM Seminar WS17/18
Algorithmic Design - Least squares estimates weighted by ex-ante return variance (WLS-EV)
"""

__author__ = 'Tobias Kuhlmann'

# Import own libraries
from variance_estimation import ExAnteVariance
from wlsev_model import Wlsev_model
from ols_model import OLS_model
import visualisation
from simon_ols_model import OLS
# import general packages
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')


/Users/tobias/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Data Preprocessing

### Read in log return data and variance

### Log returns

In [2]:
# Import price data and calc log returns
# --------------------------------------------------
es_50_prices = pd.read_csv('data/eurostoxx50_prices_eod.csv', parse_dates=True)
# set index, rename and check
es_50_prices['loctimestamp'] =pd.to_datetime(es_50_prices['loctimestamp'])
es_50_prices = es_50_prices.rename(columns={'loctimestamp': 'date'})
es_50_prices = es_50_prices.set_index('date')

#Log Returns
es_50_logret = es_50_prices
es_50_logret['logreturns'] = (np.log(es_50_prices['lastprice'] / es_50_prices['lastprice'].shift(1))).dropna()

### Volatility data

In [3]:
# Import vol data
# --------------------------------------------------
es_50_vol = pd.read_csv('data/es50_volatility.csv', parse_dates=True)
# Transform dates
es_50_vol['loctimestamp'] = pd.to_datetime(es_50_vol['loctimestamp'])
# Delete unnecessary columns
del es_50_vol['instrumentid']
# Calculate variance from vol
es_50_vol['volatility'] = es_50_vol['volatility'] ** 2
# set index, rename and check
es_50_vol = es_50_vol.rename(columns={'loctimestamp': 'date'})
es_50_vol = es_50_vol.set_index('date')

### Implied volatility data 

In [4]:
# Import implied volatility
# --------------------------------------------------
es_50_imp_vol = pd.read_csv('data/es50_implied_volatility.csv', parse_dates=True)
# Transform dates
es_50_imp_vol['loctimestamp'] = pd.to_datetime(es_50_imp_vol['loctimestamp'])
# Delete unnecessary columns
del es_50_imp_vol['instrumentid']
del es_50_imp_vol['maturity']
# Calculate implied variance from implied vol
es_50_imp_vol['implied_vol'] = es_50_imp_vol['measure'] ** 2
# set index, rename and check
es_50_imp_vol = es_50_imp_vol.rename(columns={'loctimestamp': 'date'})
es_50_imp_vol = es_50_imp_vol.set_index('date')

### Riskfree rate

In [5]:
# Import riskfree rate data
# --------------------------------------------------
rf = pd.read_csv('data/riskfree_rate.csv', parse_dates=True, sep=';')
# Transform dates
rf['loctimestamp'] = pd.to_datetime(rf['loctimestamp'])
# set index, rename and check
rf = rf.rename(columns={'loctimestamp': 'date'})
rf = rf.set_index('date')

### VRP data

In [6]:
# Import VRP data
# --------------------------------------------------
es_50_vrp = pd.read_csv('data/es50_vrp.csv', parse_dates=True)
# Transform dates
es_50_vrp['loctimestamp'] = pd.to_datetime(es_50_vrp['loctimestamp'])
# set index, rename and check
es_50_vrp = es_50_vrp.rename(columns={'loctimestamp': 'date'})
es_50_vrp = es_50_vrp.set_index('date')

### ERP data

In [7]:
# Calculate ERP from logrets and riskfree rate
# Take risk free rate maturity 7 (smallest maturity)
rf_mat7 = rf[rf['daystomaturity'] == 7]
# Calculate ERP = logrets - rf
es_50_erp = (es_50_logret['logreturns'] - rf_mat7['riskfree']).dropna()

### Q-Moments data

In [8]:
# Import Q-Moments data
# --------------------------------------------------
es_50_q = pd.read_csv('data/FiglewskiStandardizationEOD_DE0009652396D1_Qmoments.csv', parse_dates=True, sep = ';')
es_50_q.head(5)
# Transform dates
es_50_q['loctimestamp'] = pd.to_datetime(es_50_q['loctimestamp'])
# set index, rename and check
es_50_q = es_50_q.rename(columns={'loctimestamp': 'date'})
es_50_q = es_50_q.set_index('date')

# Delete unnecessary columns
del es_50_q['underlyingprice']
del es_50_q['underlyingforwardprice']
del es_50_q['Q_cubic']
del es_50_q['Q_quartic']

# Split maturities into seperate columns
es_50_q_7 = es_50_q[es_50_q['daystomaturity'] == 7]
es_50_q_30 = es_50_q[es_50_q['daystomaturity'] == 30]
es_50_q_60 = es_50_q[es_50_q['daystomaturity'] == 60]
es_50_q_91 = es_50_q[es_50_q['daystomaturity'] == 91]
es_50_q_182 = es_50_q[es_50_q['daystomaturity'] == 182]
es_50_q_365 = es_50_q[es_50_q['daystomaturity'] == 365]



### P-Moments data

In [19]:
# Import 5 min price data and calc log returns
# --------------------------------------------------
es_50_prices_5 = pd.read_csv('data/eurostoxx50_prices_5m.csv', parse_dates=True, sep=';')
# set index, rename and check
es_50_prices_5 = es_50_prices_5.rename(columns={'loctimestamp': 'date'})
es_50_prices_5['date'] = pd.to_datetime(es_50_prices_5['date'], errors='coerce')
es_50_prices_5 = es_50_prices_5.set_index('date')

#Log Returns
es_50_logret_5 = es_50_prices_5
es_50_logret_5['logreturns'] = np.log(es_50_prices_5['price'] / es_50_prices_5['price'].shift(1))
es_50_logret_5 = es_50_logret_5.dropna()

# Count of values per day
N = (es_50_logret_5.loc[(es_50_logret_5.index >= '2004-07-04 00:00:00') & (es_50_logret_5.index <= '2004-07-06 00:00:00')]).shape[0]

# Calculate moments after Amaya, Christoffersen, Jacobs, Vasquez (2015) - Does realized skewness predict equity returns
es_50_logret_5['logreturns_pow2'] = es_50_logret_5['logreturns'] ** 2
es_50_logret_5['logreturns_pow3'] = es_50_logret_5['logreturns'] ** 3
es_50_logret_5['logreturns_pow4'] = es_50_logret_5['logreturns'] ** 4

# group by date and sum up
es_50_P_1 = es_50_logret_5.groupby(es_50_logret_5.index.date).sum()

# Var 1 day = sum of intraday squared returns
es_50_P_1 = es_50_P_1.rename(columns={'logreturns_pow2': 'var'})
# Skewness 1 day
es_50_P_1['skewness'] = ( np.sqrt(N) * es_50_P_1['logreturns_pow3']) / (es_50_P_1['var'] ** (3 / 2) )
# Kurtosis 1 day
es_50_P_1['kurtosis'] = N * es_50_P_1['logreturns_pow4'] / (es_50_P_1['var'] ** 2)

# Var 7 days = sum of intraday squared returns
es_50_P_7 = es_50_P_1.rolling(7).sum()
# Skewness 7 days
es_50_P_7['skewness'] = (np.sqrt(N*7) * es_50_P_1['logreturns_pow3'].rolling(7).sum()) / (es_50_P_7['var'] ** (3 / 2) )
# Kurtosis 7 days
es_50_P_7['kurtosis'] = (N* 7 * es_50_P_1['logreturns_pow4'].rolling(7).sum()) / (es_50_P_1['var'] ** 2)

# Var 30 days = sum of intraday squared returns
es_50_P_30 = es_50_P_1.rolling(30).sum()
# Skewness 7 days
es_50_P_30['skewness'] = (np.sqrt(N * 30) * es_50_P_1['logreturns_pow3'].rolling(30).sum()) / (es_50_P_30['var'] ** (3 / 2) )
# Kurtosis 7 days
es_50_P_30['kurtosis'] = (N* 30 * es_50_P_1['logreturns_pow4'].rolling(30).sum()) / (es_50_P_30['var'] ** 2)

# Var 60 days = sum of intraday squared returns
es_50_P_60 = es_50_P_1.rolling(60).sum()
# Skewness 7 days
es_50_P_60['skewness'] = (np.sqrt(N * 60) * es_50_P_1['logreturns_pow3'].rolling(60).sum()) / (es_50_P_60['var'] ** (3 / 2) )
# Kurtosis 7 days
es_50_P_60['kurtosis'] = (N* 60 * es_50_P_1['logreturns_pow4'].rolling(60).sum()) / (es_50_P_60['var'] ** 2)



del es_50_P_1['price']
del es_50_P_1['logreturns']
del es_50_P_1['logreturns_pow3']
del es_50_P_1['logreturns_pow4']

print(es_50_P_7)

/Users/tobias/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tobias/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tobias/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

                 price  logreturns       var  logreturns_pow3  \
2000-06-29         NaN         NaN       NaN              NaN   
2000-06-30         NaN         NaN       NaN              NaN   
2000-07-03         NaN         NaN       NaN              NaN   
2000-07-04         NaN         NaN       NaN              NaN   
2000-07-05         NaN         NaN       NaN              NaN   
2000-07-06         NaN         NaN       NaN              NaN   
2000-07-07  4877149.04    0.003963  0.000586     1.238573e-07   
2000-07-10  4902381.04    0.039197  0.000528     5.048774e-07   
2000-07-11  4913945.92    0.016572  0.000498     2.818514e-07   
2000-07-12  4926161.61    0.011909  0.000482     2.459086e-07   
2000-07-13  4937350.13    0.021682  0.000482     2.857081e-07   
2000-07-14  4953743.46    0.029337  0.000436     3.456632e-07   
2000-07-17  4980942.13    0.038981  0.000400     3.938616e-07   
2000-07-18  4997373.79    0.011981  0.000351     8.229788e-08   
2000-07-19  5002406.38   

### Fama-French data

In [ ]:
# Import Fama French Factors
# --------------------------------------------------
# HML and SMB
es_50_ff = pd.read_csv('data/FamaFrench_Europe_3_Factors_Daily.csv', parse_dates=True, skiprows=6)
es_50_ff = es_50_ff.rename(columns={'Unnamed: 0': 'date'})
es_50_ff['date'] = pd.to_datetime(es_50_ff['date'], format = '%Y%m%d')
es_50_ff = es_50_ff.set_index('date')

# Momentum Factor
es_50_ff2 = pd.read_csv('data/FamaFrench_Europe_MOM_Factor_Daily.csv', parse_dates=True, skiprows=6)
es_50_ff2 = es_50_ff2.rename(columns={'Unnamed: 0': 'date'})
es_50_ff2['date'] = pd.to_datetime(es_50_ff2['date'], format = '%Y%m%d')
es_50_ff2 = es_50_ff2.set_index('date')

# Join and drop na's
es_50_ff = es_50_ff.join(es_50_ff2).dropna()



## Join data for correct dates

In [ ]:
# join vol and implied vol
print(es_50_logret['logreturns'].shape)
print(es_50_vol.shape)
print(es_50_imp_vol.shape)
print(es_50_vrp.shape)
print(es_50_erp.shape)
print(es_50_q.shape)
print(es_50_P.shape)
print(es_50_ff.shape)



# Model and Analysis

## Estimate Ex ante Variance

In [ ]:
# Model and Analysis
# ==================================================
#
# 1. Estimate (sigma_t)2, the (ex ante) conditional variance of next-period unexpected returns epsilon_(t+1)
# using a HAR-RV (Hierachical Autoregressive-Realized Variance) Model from Corsi (2009)
# ------------------------------------------------------------------------------------------------------------
# First, instantiate object
# no implied vol
ea_var_obj = ExAnteVariance(es_50_vol)
# implied vol exists
# ea_var_obj = ExAnteVariance(es_50_imp_vol, es_50_imp_vol['implied_vol'])

# Estimate Variance
result = ea_var_obj.estimate_variance()
result = result.dropna()

# Join returns and estimated variance
wlsev_var_rets = es_50_logret.join(result).dropna()


## WLS-EV and benchmark estimations

### Regress returns on returns

##### Forecast horizon 1, 5 and 10

In [ ]:
# 2. least squares estimates weighted by ex-ante return variance (WLS-EV) using Johnson (2016)
# ------------------------------------------------------------------------------------------------------------

for i in (1,5,10):
    # set forecast_horizon
    forecast_horizon = i

    # WLS-EV
    wlsev_obj = Wlsev_model(wlsev_var_rets['logreturns'][:-1].as_matrix(), wlsev_var_rets['logreturns'][1:].as_matrix(), wlsev_var_rets['vol_daily_est'][:-1].as_matrix(), forecast_horizon)
    wlsev_obj.fit()
    # OOS evaluation to get Rsquared
    wlsev_obj.evaluate()
    wlsev_obj.print_results()
    wlsev_obj.plot_results()
    # get data
    X, Y, y_wlsev = wlsev_obj.get_plot_data_wlsev()

    # OLS
    ols_obj = OLS_model(wlsev_var_rets['logreturns'][:-1].as_matrix(), wlsev_var_rets['logreturns'][1:].as_matrix(), forecast_horizon)
    ols_obj.fit()
    # OOS evaluation to get Rsquared
    ols_obj.evaluate()
    ols_obj.print_results()
    ols_obj.plot_results()
    # get data
    X, Y, y_ols = ols_obj.get_plot_data_ols()


    # Visualisation
    # ------------------------------------------------------------------------------------------------------------

    # time series plot
    visualisation.plot_results(X,Y,y_wlsev, y_ols)
    # scatter plot
    visualisation.plot_scatter(X,Y,y_wlsev, y_ols)


    # Get Simon's OLS estimation results
    if forecast_horizon == 1:
        ols_model = OLS(wlsev_var_rets['logreturns'][:-1].as_matrix(), wlsev_var_rets['logreturns'][1:].as_matrix())
        ols_model.fit()
        ols_model.printResults()

### Regress returns on VRP

##### Join vrp data with wls-ev log rets and ex ante variance

In [ ]:
es_50_vrp_rets_var = wlsev_var_rets.join(es_50_vrp).dropna()

##### forecast horizon months

In [ ]:
# set forecast_horizon
for i in (1,22,44, 66, 88):
    forecast_horizon = i

    # WLS-EV
    wlsev_obj = Wlsev_model(es_50_vrp_rets_var['vrp'][:-1].as_matrix(), es_50_vrp_rets_var['logreturns'][1:].as_matrix(), es_50_vrp_rets_var['vol_daily_est'][:-1].as_matrix(), forecast_horizon)
    wlsev_obj.fit()
    # OOS evaluation to get Rsquared
    wlsev_obj.evaluate()
    wlsev_obj.print_results()
    wlsev_obj.plot_results()
    # get data
    X, Y, y_wlsev = wlsev_obj.get_plot_data_wlsev()

    # OLS
    ols_obj = OLS_model(es_50_vrp_rets_var['vrp'][:-1].as_matrix(), es_50_vrp_rets_var['logreturns'][1:].as_matrix(), forecast_horizon)
    ols_obj.fit()
    # OOS evaluation to get Rsquared
    ols_obj.evaluate()
    ols_obj.print_results()
    # get data
    X, Y, y_ols = ols_obj.get_plot_data_ols()


    # Visualisation
    # ------------------------------------------------------------------------------------------------------------

    # time series plot
    visualisation.plot_results(X,Y,y_wlsev, y_ols)
    # scatter plot
    visualisation.plot_scatter(X,Y,y_wlsev, y_ols)

    # Get Simon's OLS estimation results
    if forecast_horizon == 1:
        ols_model = OLS(es_50_vrp_rets_var['vrp'][:-1].as_matrix(), es_50_vrp_rets_var['logreturns'][1:].as_matrix())
        ols_model.fit()
        ols_model.printResults()
        ols_obj.plot_results()

### Regress ERP on VRP

### Regress P-Moments on Q-Moments


### Regress Fama French Factors on Q-Moment